In [1]:
import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import base64
import papermill as pm
import os 
import sys
import re 
import datetime
import pickle
import nbconvert

In [2]:
import nbformat
from traitlets.config import Config
from nbconvert import HTMLExporter
from nbconvert.writers import FilesWriter

## Input parameters

In [3]:
db_location = 'data/listening_history.db'

## Run Notebooks

In [4]:
today_str = str(datetime.datetime.now().date())

In [5]:
tm_notebook = "get_latest_songs_prod.ipynb"
tm_new_notebook = f"run_notebooks/get_latest_songs_prod_{today_str}.ipynb"

In [11]:
try:
    pm.execute_notebook(tm_notebook
              , tm_new_notebook
              , {"db_location": db_location})
    tm_error = None
except Exception as e:
    tm_error = str(e)
    print("Error")
    print(tm_error)

## Email

In [12]:
def wrap_as_html(new_notebook_name):
    
    with open(new_notebook_name, 'r') as hnd:
        notebook_txt = hnd.read()
    nb_notebook = nbformat.reads(notebook_txt, as_version=4)
    html_exporter = HTMLExporter()
    html_exporter.template_file = 'basic'
    (body, resources) = html_exporter.from_notebook_node(nb_notebook)

    writer1 = FilesWriter()
    html_name = re.sub('.ipynb', '', new_notebook_name)
    writer1.write(output = body, resources = resources, notebook_name=html_name)
    html_name = html_name + '.html'


    return(html_name)

In [13]:
def create_html_attachment(html_name):
    
    with open(html_path, 'r') as hnd:
        html = hnd.read()
    part1 = MIMEText(html, 'html')
    part1.add_header("Content-Disposition", f"attachment; filename= {html_path}")
    
    return(part1)

In [14]:
gmail_login = "malctaylordev@gmail.com"
sender_email = "malctaylordev@gmail.com"
reciever_email = "malctaylor15@gmail.com"
# reciever_email = gmail_login

In [15]:
with open('data/email_pw.pkl', 'rb') as hnd:
    pw = pickle.load(hnd)['pw']

In [16]:
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = reciever_email
port = 465
context = ssl.create_default_context()

In [17]:
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = reciever_email
port = 465
context = ssl.create_default_context()

In [18]:
if isinstance(tm_error, str):
    print("Error workflow...")
    subject = "Error in Spotify runner_"+today_str
    message["Subject"] = subject

    
    body = """
        There was an error in Scraping hourly Spotify data. 

        Stack Trace: 

        """ + tm_error
    
    text = MIMEText(body, "plain")
    
    message.attach(text)

    try:
        html_path = wrap_as_html(tm_new_notebook)
        html_attachment = create_html_attachment(html_path)
        message.attach(html_attachment)

    except Exception as e: 
        print(e)
        body = body + " \n\n Error attaching notebook... WIP \n\n" + str(e)
  
    
    with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
        server.login(gmail_login, pw)
        server.sendmail(gmail_login, reciever_email, message.as_string())
else: 
    os.system("rm "+ tm_new_notebook)
    print("removed notebook: ", tm_new_notebook)

removed notebook:  run_notebooks/get_latest_songs_prod_2020-08-02.ipynb
